In [0]:
df_npis=spark.read.parquet('/mnt/bronze/npi_extract/*.parquet')

display(df_npis)

df_npis.createOrReplaceTempView('npis')

In [0]:
%sql
CREATE TABLE IF NOT EXISTS healthcarerevenuecyclemanagement_databricks.silver.npi_extracts(
  npi_id STRING,
  first_name STRING,
  last_name STRING,
  last_updated DATE,
  organisation_name STRING,
  position STRING,
  refreshed_at DATE,
  inserted_date DATE,
  updated_date DATE,
  is_current BOOLEAN
) USING DELTA;

In [0]:
%sql
MERGE INTO healthcarerevenuecyclemanagement_databricks.silver.npi_extracts AS target
USING npis AS source
ON target.npi_id = source.npi_id AND target.is_current = True
WHEN MATCHED AND (
  target.first_name <> source.first_name OR
  target.last_name != source.last_name OR
  target.position != source.position OR
  target.organisation_name != source.organisation_name OR
  target.last_updated != source.last_updated
) THEN
  UPDATE SET
  target.is_current = FALSE,
  target.updated_date = current_date()
WHEN NOT MATCHED THEN
  INSERT (
  npi_id,
  first_name,
  last_name,
  last_updated,
  organisation_name,
  position,
  refreshed_at,
  inserted_date,
  updated_date,
  is_current
  )
  VALUES (
  source.npi_id,
  source.first_name,
  source.last_name,
  source.last_updated,
  source.organisation_name,
  source.position,
  source.refreshed_at,
  current_date(),
  current_date(),
  True
  )

In [0]:
%sql
SELECT * FROM healthcarerevenuecyclemanagement_databricks.silver.npi_extracts;